In [1]:
import os, json

with open('conf.json', 'r') as f:
    json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['openai_config']['API_KEY']

In [2]:
from langchain.document_loaders import JSONLoader

def metadata_func(record: dict, metadata: dict) -> dict:
        
    for k, v in record.items():
        if type(record.get(k)) not in [str, int, float]:
            continue
        
        metadata[k] = record.get(k)

    return metadata

product_loader = JSONLoader('./product.json',
                    jq_schema=".content[]",
                    content_key="product",
                    metadata_func=metadata_func
                    )

biz_loader = JSONLoader('./symbol.json',
                    jq_schema=".content[]",
                    content_key="biz",
                    metadata_func=metadata_func
                    )

name_loader = JSONLoader('./companyName.json',
                    jq_schema=".content[]",
                    content_key="companyName",
                    metadata_func=metadata_func
                    )

kwd_loader = JSONLoader('./symbol.json',
                    jq_schema=".content[]",
                    content_key="kwd",
                    metadata_func=metadata_func
                    )

sector_loader = JSONLoader('./sector.json',
                    jq_schema=".content[]",
                    content_key="sector",
                    metadata_func=metadata_func
                    )

industry_loader = JSONLoader('./industry.json',
                    jq_schema=".content[]",
                    content_key="industry",
                    metadata_func=metadata_func
                    )

# product_docs = product_loader.load()
# biz_docs = biz_loader.load()
# kwd_docs = kwd_loader.load()
# sector_docs = sector_loader.load()
# industry_docs = industry_loader.load()
# companyName_docs = name_loader.load()

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

def create_vectordb(dirname, docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200) #10000자 씩 끊되, 200자 씩 겹치게 만든다.
    texts = text_splitter.split_documents(docs)
    
    persist_directory=dirname

    embedding = OpenAIEmbeddings(
        model='text-embedding-ada-002'
    )

    vectordb = Chroma.from_documents(
        documents=texts,
        embedding=embedding,
        persist_directory=persist_directory,
    )
    
    return vectordb

def load_vectordb(dirname):
    persist_directory = dirname

    embedding = OpenAIEmbeddings(
        model='text-embedding-ada-002'
    )

    vectordb = Chroma(  # 기존 벡터 DB 로드
        persist_directory=persist_directory,
        embedding_function=embedding
    )

    return vectordb

# prod_db = create_vectordb(dirname="prod_db", docs=product_docs)
# biz_db = create_vectordb(dirname="biz_db", docs=biz_docs)
# kwd_db = create_vectordb(dirname="kwd_db", docs=kwd_docs)
# sector_db = create_vectordb(dirname="sector_db", docs=sector_docs)
# industry_db = create_vectordb(dirname="industry_db", docs=industry_docs)

prod_db = load_vectordb(dirname="prod_db")
biz_db = load_vectordb(dirname="biz_db")
kwd_db = load_vectordb(dirname="kwd_db")
sector_db = load_vectordb(dirname="sector_db")
industry_db = load_vectordb(dirname="industry_db")
name_db = load_vectordb(dirname="name_db")

In [4]:
import pandas as pd

def get_sim_df(keyword, db, prefix, k):
    docs = db.similarity_search_with_score(keyword, k=k)
    
    docs_list = [doc[0].metadata for doc in docs]
    sims_list = [doc[1] for doc in docs]

    df = pd.DataFrame(docs_list)
    df[f"{prefix}Sim"] = sims_list
    
    return df.drop(["seq_num", "source"], axis=1)

In [177]:
keyword = "Investment"

product_df = get_sim_df(keyword=keyword, db=prod_db, prefix="product", k=7000)

product_df = product_df[product_df["product"]!="Service"]
product_df = product_df[product_df["product"]!="Product"]

biz_df = get_sim_df(keyword=keyword, db=biz_db, prefix="biz", k=7000)
kwd_df = get_sim_df(keyword=keyword, db=kwd_db, prefix="kwd", k=7000)
sector_df = get_sim_df(keyword=keyword, db=sector_db, prefix="sector", k=100)
industry_df = get_sim_df(keyword=keyword, db=industry_db, prefix="industry", k=50)
name_df = get_sim_df(keyword=keyword, db=name_db, prefix="name", k=2000)

Number of requested results 7000 is greater than number of elements in index 2402, updating n_results = 2402
Number of requested results 7000 is greater than number of elements in index 2376, updating n_results = 2376
Number of requested results 100 is greater than number of elements in index 11, updating n_results = 11


In [178]:
df = pd.merge(left=product_df, right=biz_df[["symbol", "bizSim"]], how="left", on="symbol")
df = pd.merge(left=df, right=name_df, how="left", on="companyName")
df = pd.merge(left=df, right=kwd_df[["symbol", "kwdSim"]], how="left", on="symbol")
df = pd.merge(left=df, right=sector_df, how="left", on="sector")
df = pd.merge(left=df, right=industry_df, how="left", on="industry")

df = df.dropna()

df["marketCap"] = round(df["marketCap"]/1e9, 2)
df["productRevenueRatio"] = round(df["productRevenue"]/df["totalRevenue"], 2)
df["productRevenue"] = round(df["productRevenue"]/1e9, 2)

df = df[["symbol", "companyName", "marketCap", "productRevenue", "productSim","bizSim", "nameSim", "kwdSim", "sectorSim", "industrySim", "product", "biz", "kwd", "sector", "industry", "productRevenueRatio"]]

threshhold = 0.3
biz_threshold = 0.35

df = df[(df["productSim"]<threshhold) | (df["bizSim"]<biz_threshold) | (df["kwdSim"]<biz_threshold) | (df["industrySim"]<threshhold) | (df["sectorSim"]<threshhold)].sort_values(by="marketCap", ascending=False).reset_index(drop=True)

df["industryRatio"] = round(df["productRevenue"]/df["productRevenue"].sum(), 2)

def get_duplicated_df(df):

    symbol_df = df.groupby(by="symbol")

    df_list = []

    for symbol, df_ in symbol_df:
        
        df_ = df_.sort_values(by="productSim")

        tmp_df = df_.iloc[0].copy()

        tmp_df["product"] = ", ".join(df_["product"].to_list())
        tmp_df["productRevenue"] = df_["productRevenue"].sum()
        tmp_df["productSim"] = df_["productSim"].min()
        tmp_df["industryRatio"] = df_["industryRatio"].sum()
        tmp_df["productRevenueRatio"] = df_["productRevenueRatio"].sum()

        df_list.append(tmp_df)
        
    return pd.concat(df_list, axis=1).T

final_df = get_duplicated_df(df=df)

In [179]:
final_df = final_df.sort_values(by="marketCap", ascending=False).reset_index(drop=True)

final_df

,symbol,companyName,marketCap,productRevenue,productSim,bizSim,nameSim,kwdSim,sectorSim,industrySim,product,biz,kwd,sector,industry,productRevenueRatio,industryRatio
0,JPM,JPMorgan Chase & Co.,421.44,47.9,0.284263,0.456795,0.422652,0.426767,0.331874,0.386917,Corporate & Investment Bank,JPMorgan Chase & Co. operates as a financial s...,"Financial Services, Financial Select Sector F...",Financial Services,Banks—Diversified,0.61,0.1
1,BAC,Bank of America Corporation,217.57,15.9,0.251272,0.445466,0.42005,0.440708,0.331874,0.386917,Investment and Brokerage Services,"Bank of America Corporation, through its subsi...","Invesco KBW Bank, Bank, V S&P Banks, Davis ...",Financial Services,Banks—Diversified,0.38,0.03
2,WFC,Wells Fargo & Company,149.5,1.44,0.210971,0.430064,0.438371,0.447785,0.331874,0.386917,Investment Advice,"Wells Fargo & Company, a diversified financial...","Invesco KBW Bank, Bank, V S&P Banks, Davis ...",Financial Services,Banks—Diversified,0.06,0.0
3,BX,Blackstone Inc.,130.36,0.53,0.194434,0.419083,0.411591,0.465056,0.331874,0.29666,Investment Performance,Blackstone Inc. is an alternative asset manage...,"Dorsey Wright Momentum & Dividend, Core Altern...",Financial Services,Asset Management,1.0,0.0
4,PLD,"Prologis, Inc.",103.67,4.95,0.376986,0.445732,0.441216,0.408706,0.291982,0.351892,"Strategic Capital Segment, Real Estate Operati...","Prologis, Inc. is the global leader in logisti...","Benchmark Industrial Real Estate SCTR, Janus H...",Real Estate,REIT—Industrial,1.0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,ONL,Orion Office REIT Inc.,0.3,0.21,0.355452,0.414814,0.406521,0.44435,0.291982,0.36762,"Rental Revenue, Fees From Unconsolidated Joint...",Orion Office REIT specializes in the ownership...,"Invesco S&P Spin-Off, Military Times Best Empl...",Real Estate,REIT—Office,1.0,0.0
175,SCU,"Sculptor Capital Management, Inc.",0.29,0.12,0.194434,0.435147,0.417158,0.471808,0.331874,0.29666,"Investment Performance, Asset Management","Sculptor Capital Management, Inc. is a publicl...","Russell 2000, Russell 2000 Covered Call, Russe...",Financial Services,Asset Management,1.0,0.0
176,BHR,Braemar Hotels & Resorts Inc.,0.23,0.67,0.37808,0.41897,0.44897,0.468762,0.291982,0.384419,Hotel,Braemar Hotels & Resorts Inc. is a real estate...,"Core REIT, Russell 2000 Value, Dimensional R...",Real Estate,REIT—Hotel & Motel,1.0,0.0
177,OPI,Office Properties Income Trust,0.2,0.32,0.260629,0.447333,0.414587,0.479327,0.291982,0.36762,Investment In Real Estate Segment,OPI is a national REIT focused on owning and l...,"$ High Yield Corp Bond GBP Hedged (Dist), $ ...",Real Estate,REIT—Office,1.0,0.0


In [180]:
last_df = pd.DataFrame()

for i, row in final_df.iterrows():
    prod_sim = row["productSim"]
    biz_sim = row["bizSim"]
    kwd_sim = row["kwdSim"]
    sector_sim = row["sectorSim"]
    industry_sim = row["industrySim"]
    
    if prod_sim < threshhold:
        row["기준"] = "매출"
        row["근거"] = row["product"]
        
    elif (kwd_sim < biz_threshold):
        row["기준"] = "시장인식"
        row["근거"] = row["kwd"]
        
    elif (sector_sim < threshhold):
        row["기준"] = "시장인식"
        row["근거"] = row["sector"]
        
    elif (industry_sim < threshhold):
        row["기준"] = "시장인식"
        row["근거"] = row["industry"]
        
    elif biz_sim < biz_threshold:
        row["기준"] = "정체성"
        row["근거"] = row["biz"]
            
    last_df = pd.concat([last_df, row], axis=1)

In [181]:
last_df = last_df.T[["symbol", "기준", "근거", "companyName"]]

last_df = last_df.rename(columns={"symbol": "심볼", "companyName": "회사명"})

last_df["테마"] = keyword

last_df

,심볼,기준,근거,회사명,테마
0,JPM,매출,Corporate & Investment Bank,JPMorgan Chase & Co.,Investment
1,BAC,매출,Investment and Brokerage Services,Bank of America Corporation,Investment
2,WFC,매출,Investment Advice,Wells Fargo & Company,Investment
3,BX,매출,Investment Performance,Blackstone Inc.,Investment
4,PLD,시장인식,Real Estate,"Prologis, Inc.",Investment
...,...,...,...,...,...
174,ONL,시장인식,Real Estate,Orion Office REIT Inc.,Investment
175,SCU,매출,"Investment Performance, Asset Management","Sculptor Capital Management, Inc.",Investment
176,BHR,시장인식,Real Estate,Braemar Hotels & Resorts Inc.,Investment
177,OPI,매출,Investment In Real Estate Segment,Office Properties Income Trust,Investment


In [182]:
last_df.to_csv(f"./mgics_theme/{keyword.lower().replace(' ', '_')}.csv")

### FINAL
- 14개 테마 통합

In [187]:
folder_path = './mgics_theme'  # .csv 파일이 저장된 폴더 경로를 지정하세요.

# .csv 파일을 저장할 빈 데이터프레임을 생성합니다.
combined_df = pd.DataFrame()

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, index_col=0)  # 각 .csv 파일을 데이터프레임으로 읽기
        combined_df = pd.concat([combined_df, df], ignore_index=True)  # 데이터프레임 합치기

# combined_df에 모든 .csv 파일의 데이터가 합쳐집니다.

In [191]:
combined_df.to_csv(f"./mgics_theme/combined_df.csv")

- DB에 종목데이터들 적재

In [195]:
symbol_list = list(set(combined_df["심볼"].to_list()))

- 종목 지난 1년간 가격 크롤링

In [199]:
import bt
from datetime import datetime
from dateutil.relativedelta import relativedelta

now = datetime.now()
a_year_ago = now - relativedelta(years=1)

data = bt.get(",".join(symbol_list), start=a_year_ago)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [204]:
data = pd.read_csv("./price_231007.csv", index_col="Date")
data.head()

,agro,ftnt,frme,hbanp,px,cma,cuz,mx,cash,out,...,mmi,bhfam,tuya,hr,mtb,gnl,adsk,rgld,sho,five
Date,,,,,,,,,,,,,,,,,,,,,
2022-10-14,7.856009,48.130001,40.382736,16.332628,10.206156,67.928627,21.424341,10.12,37.389492,14.715453,...,34.007351,14.821254,0.973,17.879883,178.919083,9.500975,189.809998,88.387932,9.808957,131.759995
2022-10-17,8.115951,50.459999,40.959499,16.529518,10.196275,69.398163,22.056686,10.14,38.106415,15.296692,...,34.912903,14.824040,0.998,18.395830,181.175339,9.827344,198.699997,90.500641,10.189982,136.160004
2022-10-18,8.096696,51.560001,41.113300,16.501394,10.354357,70.029320,21.952868,9.88,37.897316,16.025545,...,35.090076,14.914120,1.030,18.489635,182.583099,9.963331,200.699997,92.287552,10.307220,140.360001
2022-10-19,8.404775,51.790001,41.219040,16.473265,10.304955,63.727238,20.876932,9.69,37.339703,15.453532,...,33.790806,14.514799,0.990,17.842358,157.224319,9.800147,197.020004,89.641739,10.277912,135.270004
2022-10-20,8.298873,51.590000,40.584602,16.257624,10.196275,61.004826,21.084568,9.68,36.164749,15.195206,...,33.751434,14.421934,0.980,17.579695,155.623749,9.646029,197.830002,90.411781,10.092283,131.210007
